In [12]:
import torch
from torch import nn
import torchmetrics

import wandb

import os
from glob import glob
import json

import torch
from torch.utils.data import Dataset

import rasterio

import numpy as np

import segmentation_models_pytorch as smp

import timm

/opt/homebrew/Caskroom/miniconda/base/envs/flair-2-env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


# Confusion matrix WandB

In [2]:
wandb.init(
    entity='urgellbapt',
    project='developpement',
    group='ConfusionMatrix',
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: urgellbapt. Use `wandb login --relogin` to force relogin


In [3]:
class ConfusionMatrix(torchmetrics.Metric):
    def __init__(self, classes) -> None:
        super().__init__()
        
        self.target = []
        self.preds = []
        self.classes = classes
    
    def update(self, preds, target):
        self.target.append(target)
        self.preds.append(preds)
    
    def compute(self):
        target = torch.cat(self.target, dim=0)
        target = torch.flatten(target)
        target = target.tolist()
        
        preds = torch.cat(self.preds, dim=0)
        # preds = torch.argmax(preds, dim=3)
        preds = torch.flatten(preds, start_dim=0, end_dim=2)
        preds = preds
        
        return wandb.plot.confusion_matrix(probs=preds, y_true=target, class_names=self.classes)
        
        
    def reset(self):
        self.target = []
        self.preds = []

In [4]:
a = torch.randn(1, 4, 4, 9)
abis = torch.randint(low=0, high=9, size=(1, 4, 4))
b = torch.randn(1, 4, 4, 9)
bbis = torch.randint(low=0, high=9, size=(1, 4, 4))
softmax = torch.nn.Softmax(dim=3)
a = softmax(a)
b = softmax(b)

metric = ConfusionMatrix(classes=[f"test{i}" for i in range(9)])
metric.update(a, abis)
metric.update(b, bbis)
wandb.log({'confusion_matrix': metric.compute()})
metric.reset()
display(metric.target)

[]

In [5]:
wandb.finish()

# Model

## Pytorch

In [15]:
model_params = {
    'encoder_name': "efficientnet-b5",
    'encoder_weights': None,
    'in_channels': 3,
    'activation': "softmax",
    'classes': 13
}

model = smp.Unet(
    **model_params
)

In [17]:
tensor = torch.randn(1, 3, 256, 256)
model(tensor).shape

/opt/homebrew/Caskroom/miniconda/base/envs/flair-2-env/lib/python3.10/site-packages/segmentation_models_pytorch/base/modules.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


torch.Size([1, 13, 256, 256])

In [7]:
# Example of target with class indices
criterion_weight = torch.as_tensor([1]* 5, dtype=torch.float32)
loss = nn.CrossEntropyLoss(weight=criterion_weight)
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input.dtype, target.dtype)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input, target)
output = loss(input, target)
output.backward()

torch.float32 torch.int64
tensor([[-0.6487, -0.0743,  0.7115,  0.6750, -0.3831],
        [ 1.0901, -0.2194, -0.6861, -1.1891, -0.3441],
        [ 0.6746,  1.3344, -0.5492, -0.4880,  0.9297]], requires_grad=True) tensor([[0.0626, 0.0890, 0.3158, 0.4509, 0.0817],
        [0.0737, 0.1408, 0.0396, 0.2601, 0.4858],
        [0.1004, 0.0194, 0.2513, 0.4363, 0.1926]])


# Tiff files

In [1]:
import tifffile
img = tifffile.imread('../data/raw/train/aerial/D004_2021/Z1_NN/img/IMG_000001.tif')

In [9]:
import torch
tensor = torch.tensor(img).to(device='mps')
img = tensor.numpy(force=True)
type(img)
tifffile.imwrite(f'PRED_{1}.tif', img)

In [10]:
tensor = torch.randn(4, 3, 30, 30)
for sub_tensor in tensor:
    print(sub_tensor.shape)

torch.Size([3, 30, 30])
torch.Size([3, 30, 30])
torch.Size([3, 30, 30])
torch.Size([3, 30, 30])
